# Análisis Predictivo de Resultados de Fútbol

Este proyecto implementa un sistema para predecir resultados de partidos de fútbol utilizando datos históricos de la liga española de la web de AS. Emplea expresiones regulares para la extracción, One-Hot Encoding para el preprocesamiento de equipos y, finalmente, un modelo de Regresión de Bosques Aleatorios (Random Forest Regressor) para el entrenamiento y la predicción.

David Ramajo Fernández

## 1. Extracción de datos (Scraping)

Se utilizan expresiones regulares (regex) específicas para extraer los nombres de los equipos y los marcadores de los partidos, adaptándose a los diferentes formatos de la página a lo largo de los años.

Los datos se limpian y se estructuran en listas de equipos, años y goles.

In [ ]:
# paso 1: obtener la página
# abrir en chrome, y buscar la url de los resultados de la liga
# en este caso se usa los resultados del as, por lo que se busca la primera jornada disponible en el as

url = "https://resultados.as.com/resultados/futbol/primera/2001_2002/calendario"


In [ ]:
import urllib.request
import ssl
import certifi

def get_url(url):
    req = urllib.request.Request(
        url,
        data=None,
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        }
    )
    f = urllib.request.urlopen(req, context=ssl.create_default_context(cafile=certifi.where()))
    content = f.read().decode()
    f.close()
    return content

print(get_url(url))

<!DOCTYPE html>
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<title>Calendario Primera división 2001/2002 en AS.com</title>
<meta name="Description" content="Consulta aquí el calendario de la Primera División de Fútbol Española 2001/2002" />
<meta name="Keywords" content="calendario, resultados. partidos, estadisticas, directos, clasificacion, Primera división 2001/2002, Liga de primera división de fútbol española" />
<link href="//resultados.as.com/resultados/futbol/primera/2001_2002/calendario/regular_a" rel="canonical">
<meta name="Generator" content="Diarioas AS S.L." />
<meta name="Origen" content="As" />
<meta name="Author" content="Diarioas AS S.L." />
<meta name="Locality" content="Madrid, España" />
<meta name="Language" content="es" />
<meta name="revisit-after" content="1 days" />
<meta name="robots" content="index, follow" />
<link rel="stylesheet" href="//as01.epimg.net/css/v1.x/

In [ ]:
# paso 2: construir expresión regular. Como la paginá cambió en 2014, se tiene que construir la expresion regular para
# antes y para despues. Se empeza con antes de 2014. Los años 2004, 2005, 2010, 2012 y 2013 no se tendran en cuenta
# por contener errores (en el 2012 no aparecen los equipos) y ser necesaria
# una expresión distinta (para cada uno de los otros años).

import re
def get_mat1(content):
    matches = [re.findall('<span itemprop="name">(.*?)<',content),
              re.findall('class="resultado resul_post">(.*?) - (.*?)<',content)]
    return matches


In [ ]:
# paso 3: dar formato al contenido obtenido

datos = [[], [], []]
resultados = [[], []]
def set_forma(matches, age, resultados):
    test = len(matches[0]) - len(matches[1]*2)
    for i in range(len(matches[0])):
        if i < len(matches[1]*2):
            if age == 21 and matches[0][i] == "Barcelona" and matches[0][i+1] == "Rayo":  # El partido no se jugó
                del matches[0][i+1]
                del matches[0][i]
        if (i%2) == 0:
            if i < len(matches[0]):
                datos[0].append(matches[0][i])
                datos[2].append(age)
            if i < len(matches[1]*2):
                resultados[0].append(int(matches[1][i//2][0]))
        else:
            if i < len(matches[0]):
                datos[1].append(matches[0][i])
            if i < len(matches[1]*2):
                resultados[1].append(int(matches[1][i//2][1]))

In [ ]:
# paso 3.1: se prueba que obtenemos los datos requeridos de los primeros años correctamente.

for i in range (1, 12):
    if i == 10 or i == 4 or i == 5:
        continue
    set_forma(get_mat1(get_url(f"https://resultados.as.com/resultados/futbol/primera/20{i:02d}_20{i+1:02d}/calendario")), i, resultados)
    print(f"https://resultados.as.com/resultados/futbol/primera/20{i:02d}_20{i+1:02d}/calendario")

print(datos + resultados)

https://resultados.as.com/resultados/futbol/primera/2001_2002/calendario
https://resultados.as.com/resultados/futbol/primera/2002_2003/calendario
https://resultados.as.com/resultados/futbol/primera/2003_2004/calendario
https://resultados.as.com/resultados/futbol/primera/2006_2007/calendario
https://resultados.as.com/resultados/futbol/primera/2007_2008/calendario
https://resultados.as.com/resultados/futbol/primera/2008_2009/calendario
https://resultados.as.com/resultados/futbol/primera/2009_2010/calendario
https://resultados.as.com/resultados/futbol/primera/2011_2012/calendario
[['Valencia', 'R. Sociedad', 'Tenerife', 'Osasuna', 'Deportivo', 'Espanyol', 'Mallorca', 'Málaga', 'Rayo', 'Sevilla', 'Mallorca', 'Deportivo', 'Betis', 'Osasuna', 'Espanyol', 'R. Sociedad', 'Tenerife', 'Rayo', 'Málaga', 'Valencia', 'R. Sociedad', 'Espanyol', 'Real Madrid', 'Málaga', 'Osasuna', 'Betis', 'Tenerife', 'Valencia', 'Mallorca', 'Deportivo', 'Real Madrid', 'Deportivo', 'Betis', 'Espanyol', 'R. Sociedad',

In [ ]:
# paso 3.2: Expresion formal para la segunda parte

def get_mat2(content):
    matches = [re.findall('class="nombre-equipo">(.*?)<',content),
              re.findall('class="resultado".\n?.*?(\d+) - (\d+).*?\n.*?<',content)]
    return matches


In [ ]:
# paso 3.3: Se obtienen los datos de la segunda parte.

for i in range (14, 22):
    set_forma(get_mat2(get_url(f"https://resultados.as.com/resultados/futbol/primera/20{i:02d}_20{i+1:02d}/calendario")), i, resultados)
    print(f"https://resultados.as.com/resultados/futbol/primera/20{i:02d}_20{i+1:02d}/calendario")

print(datos + resultados)

https://resultados.as.com/resultados/futbol/primera/2014_2015/calendario
https://resultados.as.com/resultados/futbol/primera/2015_2016/calendario
https://resultados.as.com/resultados/futbol/primera/2016_2017/calendario
https://resultados.as.com/resultados/futbol/primera/2017_2018/calendario
https://resultados.as.com/resultados/futbol/primera/2018_2019/calendario
https://resultados.as.com/resultados/futbol/primera/2019_2020/calendario
https://resultados.as.com/resultados/futbol/primera/2020_2021/calendario
https://resultados.as.com/resultados/futbol/primera/2021_2022/calendario
[['Valencia', 'R. Sociedad', 'Tenerife', 'Osasuna', 'Deportivo', 'Espanyol', 'Mallorca', 'Málaga', 'Rayo', 'Sevilla', 'Mallorca', 'Deportivo', 'Betis', 'Osasuna', 'Espanyol', 'R. Sociedad', 'Tenerife', 'Rayo', 'Málaga', 'Valencia', 'R. Sociedad', 'Espanyol', 'Real Madrid', 'Málaga', 'Osasuna', 'Betis', 'Tenerife', 'Valencia', 'Mallorca', 'Deportivo', 'Real Madrid', 'Deportivo', 'Betis', 'Espanyol', 'R. Sociedad',

## 2. Preparación y modelado de datos

Los datos se cargan en Pandas para su manejo.

Los nombres de los equipos (variables categóricas) se transforman en un formato numérico usando One-Hot Encoding para que el modelo pueda procesarlos.

Los datos se dividen en un conjunto para entrenamiento/prueba (partidos jugados) y un conjunto para predicción (partidos futuros aun no jugados).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import compose
from sklearn import model_selection
from sklearn.metrics import median_absolute_error

In [ ]:
# paso 4: Se pasan los datos a pandas para mayor facilidad al buscar sus traspuestas.

x = pd.DataFrame(datos)
y = pd.DataFrame(resultados)

In [ ]:
x.transpose()

,0,1,2
0,Valencia,Real Madrid,1
1,R. Sociedad,Athletic,1
2,Tenerife,Alavés,1
3,Osasuna,Celta,1
4,Deportivo,Valladolid,1
...,...,...,...
6074,Valencia,Celta,21
6075,Elche,Getafe,21
6076,Barcelona,Villarreal,21
6077,Alavés,Cádiz,21


In [ ]:
y.transpose()

,0,1
0,1,0
1,1,3
2,0,2
3,0,3
4,4,0
...,...,...
5964,2,0
5965,1,2
5966,4,3
5967,1,3


In [ ]:
# paso 4.1: Se realiza one hot encoder a los nombres de los equipos.

column_transformer = sklearn.compose.ColumnTransformer(transformers=[
    ("one-hot", sklearn.preprocessing.OneHotEncoder(), [0,1]),
    ("pass","passthrough",[2])
]);

In [ ]:
X = column_transformer.fit_transform(x.transpose())
Y = y.transpose()

In [ ]:
X.shape

(6079, 83)

In [ ]:
x_train1 = X[:Y[0].size]

In [ ]:
x_result = X[Y[0].size:]

In [ ]:
# paso 4.2: se dividen los datos en entrenamiento y test.

x_train2, x_test, y_train, y_test = train_test_split(x_train1, Y, test_size = 0.2, random_state = 42)

## 3. Entrenamiento y Resultados

Se utiliza un modelo de Regresión de Bosques Aleatorios (Random Forest Regressor) para entrenar la predicción de resultados.

Se evalúa la precisión del modelo con los datos de test mediante el Error Absoluto Mediano.

In [ ]:
rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X = x_train2, y = y_train)
y_predicted = rfr.predict(X = x_test)

In [ ]:
median_absolute_error(y_test, y_predicted)

0.8325

## 4. Anexo

El modelo se entrena con todos los datos históricos disponibles para generar las predicciones de los partidos no jugados de la temporada.

In [ ]:
# paso 4.3: Se pasa a predecir los resultados del resto de partidos no jugados de la temporada.

rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X = x_train1, y = Y)
y_predicted = rfr.predict(X = x_result)

In [ ]:
num = y_predicted.size//2

In [ ]:
# paso 5: Se imprimen los resultados.

for i in range (0, num):
    print(x.transpose()[0][i+Y[0].size] + " " + str(y_predicted[i][0]) + " - " + str(y_predicted[i][1]) + " " + x.transpose()[1][i+Y[0].size])

Atlético 2.71 - 0.25 Cádiz
Levante 1.0 - 1.1 Espanyol
Granada 1.56 - 1.03 Elche
Villarreal 2.48 - 2.39 Celta
Getafe 2.03 - 0.51 Valencia
Rayo 0.34 - 1.19 Sevilla
Betis 1.13 - 0.69 Athletic
R. Sociedad 1.8 - 0.07 Alavés
Barcelona 3.03 - 0.81 Osasuna
Mallorca 0.67 - 1.14 Real Madrid
Athletic 3.03 - 0.9 Getafe
Alavés 2.58 - 1.49 Granada
Elche 0.9 - 1.49 Valencia
Osasuna 1.34 - 1.56 Levante
Rayo 1.09 - 1.84 Atlético
Espanyol 1.69 - 0.51 Mallorca
Cádiz 0.5 - 0.69 Villarreal
Celta 1.43 - 2.29 Betis
Sevilla 2.79 - 1.7 R. Sociedad
Real Madrid 1.73 - 0.66 Barcelona
Athletic 1.71 - 0.69 Elche
Valencia 1.37 - 0.92 Cádiz
Getafe 2.67 - 1.54 Mallorca
Levante 0.91 - 3.01 Villarreal
Atlético 1.42 - 0.38 Alavés
Celta 1.0 - 2.75 Real Madrid
Granada 1.48 - 1.1 Rayo
R. Sociedad 1.15 - 0.25 Espanyol
Betis 1.5 - 0.63 Osasuna
Barcelona 2.35 - 0.96 Sevilla
Elche 0.56 - 2.06 R. Sociedad
Villarreal 2.42 - 0.55 Athletic
Espanyol 0.73 - 0.57 Celta
Osasuna 1.33 - 0.92 Alavés
Sevilla 1.95 - 0.74 Granada
Real Madrid